># 신경망 구현하기

#### 데이터
- 샘플데이터는 6-2에서 살펴보았던 유방암 데이터 사용

In [18]:
import pandas as pd

query = 'select * from testdataset.wdbc order by index'
dataset = pd.read_gbq(project_id = 'mlwithgcp', query = query)

dataset['diagnostic'] = dataset['diagnostic'].apply(lambda X: 0 if X=='M' else 1)
dataset.drop('index', axis = 1, inplace = True)
X_dataset = dataset.drop('diagnostic', axis = 1).as_matrix()
y_dataset = dataset['diagnostic'].as_matrix()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset,
                                                   test_size = 0.2, 
                                                   random_state = 42)

Requesting query... ok.
Job ID: e4758ea9-ebda-4cb7-a051-069949e5e3d1
Query running...
Query done.
Cache hit.

Retrieving results...
Got 569 rows.

Total time taken 3.13 s.
Finished at 2018-11-06 23:55:11.


___
#### 입출력 유닛
- 샘플신경망은 2개의 레이어, 각각 4개의 유닛으로 구성
- 입력층과 출력층은 `placeholder`로 구성

In [3]:
import tensorflow as tf
import numpy as np

In [6]:
# wdbc 특징량 수 (열)
NUM_FEATURES = 30

# 특징량과 레이블 입력
X = tf.placeholder(np.float32, shape = [None, NUM_FEATURES], name = 'X')
y = tf.placeholder(np.float32, shape = [None], name = 'y')

 ___
 #### 은닉층
 - 은닉층 구현은 `tf.layers`로 간단하게 구현 가능
 - `df.layers.dense`는 전결합층<sub>Fully Connected Layer</sub>이라고 부르는 모든 유닛이 연결된 층을 의미함
 - 이를 통해 '층의 유닛수'와 '활성화 함수'만 지정하면 구현 가능
 - `df.layers.dense`는 내부적으로 `variable`을 저장하고 있으며, 학습 처리에 따라 가중치 값을 최적화
 - `df.layers.dense`를 사용하지 않는 경우 각각의 요소들을 명시적으로 정의하여 구현하는 것도 가능

In [8]:
NUM_UNITS_H1 = 4
NUM_UNITS_H2 = 4

# 은닉층
hidden1 = tf.layers.dense(inputs = X,
                         units = NUM_UNITS_H1,
                         activation = tf.nn.relu,
                         name = 'hidden1')
hidden2 = tf.layers.dense(inputs = hidden1,
                         units = NUM_UNITS_H2,
                         activation = tf.nn.relu,
                         name = 'hidden2')


#### df.layers.dense를 사용하지 않는 경우 ####
###########################################
# 은닉층1
w1 = tf.Variable(tf.truncated_normal([NUM_FEATURES, NUM_UNITS_H1], stddev = 0.1), name = 'w1')
b1 = tf.Variable(tf.zeros([NUM_UNITS_H1]), name = 'b1')
hidden1 = tf.nn.relu(tf.matmul(X, w1) + b1)

# 은닉층2
w2 = tf.Variable(tf.truncated_normal([NUM_UNITS_H1, NUM_UNITS_H2], stddev = 0.1), name = 'w2')
b2 = tf.Variable(tf.zeros([NUM_UNITS_H2]), name = 'b2')
hidden2 = tf.nn.relu(tf.matmul(X, w1) + b2)
# truncated_normal은 음수부분을 절단한 절단 정규분포로 임의의 초기값 전달을 위해 사용

- 위 코드는 별도로 유닛 수를 설정하지 않음
- 대신 가중치의 shape에 따라 행렬곱으로 복수의 유닛 계산을 수행

___
#### 출력층
- 출력값은 레이블이 2개로, 2개의 유닛으로 출력층을 구성

In [12]:
# 클래스 수
NUM_CLASSES = 2

# 출력층
logits = tf.layers.dense(inputs = hidden2,
                         units = NUM_CLASSES,
                         name = 'output')

___
#### 손실함수

- 출력결과와 정답레이블(one-hot-vector)로 손실 계산
- `softmax_cross_entropy_with_logits`로 softmax와 손실 계산을 동시에 수행

In [16]:
# 손실 (softmax + entropy)
one_hot_labels = tf.one_hot(indices = tf.cast(y, tf.int32),
                           depth = NUM_CLASSES)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels = one_hot_labels,
                                                        logits = logits,
                                                        name = 'xentropy')

loss = tf.reduce_mean(cross_entropy, name = 'xentropy_mean')

In [20]:
# 손실 최소화
train_op = tf.train.AdamOptimizer(0.01).minimize(loss)

# 정답률 계산
correct_prediction = tf.equal(tf.argmax(logits, 1),
                             tf.argmax(one_hot_labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

___
#### 계산
- 계산그래프가 완성되었고, `train_op`와 `loss`, `feed_dict:....`를 입력값으로 훈련 수행
- 1000회 반복 후 테스트 전용 데이터로 정답률 계산
- 입력층의 shape을 변경하면 다른 데이터도 학습 및 활용이 가능

In [21]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  for step in range(1000):
    _, loss_value = sess.run([train_op, loss],
                            feed_dict = {X:X_train, y:y_train})
    
    if step % 100 ==0:
      print('Step: %d, Loss: %f' % (step, loss_value))

  _a = sess.run(accuracy, feed_dict ={X:X_test, y:y_test})
  print('Accuray: %f' % _a)

Step: 0, Loss: 20.874336
Step: 100, Loss: 0.336367
Step: 200, Loss: 0.237237
Step: 300, Loss: 0.208793
Step: 400, Loss: 0.188213
Step: 500, Loss: 0.173930
Step: 600, Loss: 0.163369
Step: 700, Loss: 0.154654
Step: 800, Loss: 0.146571
Step: 900, Loss: 0.138466
Accuray: 0.956140


___
>## 정리

In [ ]:
NUM_FEATURES = 30
NUM_UNITS_H1 = 4
NUM_UNITS_H2 = 4
NUM_CLASSES = 2

with tf.Graph().as_default():
  # 유닛
  X = tf.placeholder(np.float32, shape = [None, NUM_FEATURES], name = 'X')
  y = tf.placeholder(np.float32, shape = [None], name = 'y')
  
  # 은닉층
  hidden1 = tf.layers.dense(inputs = X,
                           units = NUM_UNITS_H1,
                           activation = tf.nn.relu,
                           name = 'hidden1')
  hidden2 = tf.layers.dense(inputs = hidden1,
                           units = NUM_UNITS_H2,
                           activation = tf.nn.relu,
                           name = 'hidden2')

  # 출력층
  logits = tf.layers.dense(inputs = hidden2,
                           units = NUM_CLASSES,
                           name = 'output')

  # 손실 (softmax + entropy)
  one_hot_labels = tf.one_hot(indices = tf.cast(y, tf.int32),
                             depth = NUM_CLASSES)

  cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels = one_hot_labels,
                                                          logits = logits,
                                                          name = 'xentropy')

  loss = tf.reduce_mean(cross_entropy, name = 'xentropy_mean')


  # 손실 최소화
  train_op = tf.train.AdamOptimizer(0.01).minimize(loss)

  # 정답률 계산
  correct_prediction = tf.equal(tf.argmax(logits, 1),
                                 tf.argmax(one_hot_labels, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  
  # 실행
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
  
    for step in range(1000):
      _, loss_value = sess.run([train_op, loss],
                              feed_dict = {X:X_train, y:y_train})
    
      if step % 100 ==0:
        print('Step: %d, Loss: %f' % (step, loss_value))

    _a = sess.run(accuracy, feed_dict ={X:X_test, y:y_test})
    print('Accuray: %f' % _a)